<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [83]:
import re
import random
import numpy as np

from sklearn.datasets import fetch_20newsgroups

from keras.models import Model
from keras.layers import Embedding, Dense, Input, Dot, Multiply, Lambda
from keras.layers import dot
# from keras

In [59]:
dot?

In [12]:
WORD_RE = re.compile(r'[a-z]+')

In [3]:
data = fetch_20newsgroups()

In [4]:
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])

In [8]:
texts = list(map(lambda x: x.lower(), data['data']))

In [13]:
VOCABULARY = []
for text in texts:
    VOCABULARY += WORD_RE.findall(text)
VOCABULARY = list(set(VOCABULARY))
VOCABULARY.sort()
WORD2IND = {word: i for i, word in enumerate(VOCABULARY)}

In [28]:
INDEXES = list(range(len(VOCABULARY)))

In [91]:
word_input = Input(shape=(1, ))
word_embedding = Embedding(input_dim=len(VOCABULARY), output_dim=300)(word_input)

context_input = Input(shape=(1, ))
context_embedding = Embedding(input_dim=len(VOCABULARY), output_dim=300)(context_input)

merge = Multiply()([context_embedding, word_embedding])
t_slice = Lambda(lambda x: x[:, 0, :])(merge)
target = Dense(1, activation='sigmoid')(t_slice)

model = Model(inputs=[word_input, context_input], outputs=target)

In [92]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
WINDOW_SIZE = 5

inputs = []
contexts = []
targets = []
for text in texts:
    for sentence in text.split('.'):
        words = WORD_RE.findall(sentence)
        words_indexes = [WORD2IND[word] for word in words]
        for i in range(len(words_indexes)):
            words_sample = words_indexes[max(0, i - WINDOW_SIZE): i + WINDOW_SIZE]
            words_sample = [ind for ind in words_sample if ind != words_indexes[i]]
            
            if not words_sample:
                continue
            
            # add positive
            inputs.append([words_indexes[i]])
            contexts.append([random.choice(words_sample)])
            targets.append(1)
            
            # add negatives
            for j in range(5):
                inputs.append([words_indexes[i]])
                contexts.append([random.choice(INDEXES)])
                targets.append(0)

In [38]:
inputs = np.array(inputs)
contexts = np.array(contexts)

In [94]:
model.fit([inputs, contexts], targets)